# 8.4. 텍스트 분류 실습 - 20 뉴스그룹 분류
- 사이킷런이 내부에 가지고 있는 예제 데이터인 20 뉴스그룹 데이터 세트 이용
- 텍스트 분류: 특정 문서의 분류를 학습 데이터를 통해 학습해 모델을 생성한 뒤 이 학습 모델을 이용해 다른 문서의 분류를 예측하는 것

- 사이킷런은 fetxh_20newsgroups( ) API를 이용해 뉴스그룹의 분류를 수행해 볼 수 있는 예제 데이터를 제공
- 희소 행렬에 분류를 효과적으로 잘 처리할 수 있는 알고리즘: 로지스틱 회귀, 선형 서포트 벡터 머신, 나이브 베이즈 등

  => 로지스틱 회귀 이용

- **텍스트 기반의 분류 수행**
  1. 텍스트를 정규화
  2. 피처 벡터화 적용
  3. 적합한 머신러닝 알고리즘을 적용해 분류를 학습/예측/평가

- **주요 실습 사항**
  1. 카운트 기반과 TF-IDF 기반의 벡터화를 차례로 적용해 예측 성능을 비교
  2. 피처 벡터화를 위한 파라미터와
  GridSearchCV 기반의 하이퍼 파라미터 튜닝
  3. 사이킷런의 pipeline 객체를 통해 피처 벡터화 파라미터, GridSearchCV 기반의 하이퍼 파라미터 튜닝을 한번에 수행

## 텍스트 정규화

### 데이터 로드
- fetch_20newsgroups( ): 인터넷에서 로컬 컴퓨터로 데이터를 먼저 내려받은 후에 메모리로 데이터 로딩

In [1]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset = 'all', random_state = 156)

- fetch_20newsgroups( )는 사이킷런의 다른 데이터 세트 예제와 같이 파이썬 딕셔너리와 유사한 Bunch 객체 반환

### key 값 확인

In [ ]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


- filenames: fetch_20newsgroups( ) API가 인터넷에서 내려받아 로컬 컴퓨터에 저장하는 디렉터리와 파일명 지칭

### Target 클래스의 구성 확인

In [ ]:
import pandas as pd

print('target 클래스의 값과 분포도 \n', pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n', news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
Name: count, dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


- Target 클래스의 값: 0~19(20개)로 구성

### 개별 데이터 구성 확인

In [ ]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

- 기사 내용 + 뉴스그룹 제목, 작성자, 소속, 이메일 등의 다양한 정보
- 제목과 소속, 이메일 주소 등의 헤더와 푸터 정보들은 뉴스그룹 분류의 Target 클래스 값과 유사한 데이터를 가지고 있는 경우가 많음

  => 이 피처들을 포함하게 되면 웬만한 ML 알고리즘을 적용해도 상당히 높은 예측 성능을 나타냄

  => 내용을 제외한 다른 정보들을 제거해야함

  => remove 파라미터 이용

- fetch_20newsgroups( )는 subset 파라미터를 이용해 학습 데이터 세트와 테스트 데이터 세트를 분리해 내려받을 수 있음

### 전처리

In [2]:
from sklearn.datasets import fetch_20newsgroups

# subset = 'train'으로 학습용 데이터만 추출, remove = ('headers', 'footers', 'quotes')로 내용만 추출
train_news = fetch_20newsgroups(subset = 'train', remove = ('headers', 'footers', 'quotes'), random_state = 156)
X_train = train_news.data
y_train = train_news.target

# subset = 'test'로 학습용 데이터만 추출, remove = ('headers', 'footers', 'quotes')로 내용만 추출
test_news = fetch_20newsgroups(subset = 'test', remove = ('headers', 'footers', 'quotes'), random_state = 156)
X_test = test_news.data
y_test = test_news.target

print('학습 데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len(train_news.data), len(test_news.data)))

학습 데이터 크기 11314, 테스트 데이터 크기 7532


## 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가

### Count 기반

#### 피처 벡터화 변환
- CountVectorizer를 이용해 학습 데이터의 텍스트를 피처 벡터화
</br>

< CountVectorizer를 이용해 테스트 데이터를 피처 벡터화할 때 유의할 점 >
  - 반드시 학습 데이터를 이용해 fit( )이 수행된 CountVectorizer 객체를 이용해 테스트 데이터를 변환해야함
  - 그래야 학습 시 설정된 CountVectorizer의 피처 개수와 테스트 데이터를 CountVectorizer로 변환할 피처 개수가 같아짐
  - 테스트 데이터의 피처 벡터화는 학습 데이터에 사용된 CountVectorizer 객체 변수인 cnt_vect.transform( )을 이용해 변환

- fit_transform( ) 사용하지 않음
  - CountVectorizer.fit_transform(테스트 데이터)을 테스트 데이터 세트에 적용하면 테스트 데이터 기반으로 다시 CountVectorizer가 fit( )을 수행하고 transform( )하기 때문에 학습 시 사용된 피처 개수와 예측 시 사용할 피처 개수가 달라짐

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vetorization으로 피처 벡터화 변환 수행
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

# 학습 데이터로 fit( )된 CountVectorizer를 이용해 테스트 데이터를 피처 벡터화 변환 수행
X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터 텍스트의 CountVectorizer shape: ', X_train_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorizer shape:  (11314, 101631)


- 학습 데이터를 CountVectorizer로 피처를 추출한 결과 11314개의 문서에서 피처(단어)가 101631개로 만들어짐

#### 로지스틱 회귀를 적용해 분류를 예측

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# LogisticRegression을 이용해 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect, y_train)
pred = lr_clf.predict(X_test_cnt_vect)

print('CountVectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

CountVectorized Logistic Regression의 예측 정확도는 0.603


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### TF-IDF 기반

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 벡터화를 적용해 학습 데이터 세트와 테스트 데이터 세트 변환
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# LogisticRegression을 이용해 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.674


< 텍스트 분석에서 머신러닝 모델의 성능을 향상시키는 중요한 2가지 방법 >

1. 최적의 ML 알고리즘 선택
2. 최선의 피처 전처리 수행

## 다양한 파라미터 적용

### 스톱워드, ngram, max_df 파라미터 변경
- 스톱워드: 'None' -> 'english'
- ngram_range: (1,1) -> (1,2)
- max_df = 300

In [6]:
# stop words 필터링을 추가하고 ngram을 기본 (1, 1)에서 (1, 2)로 변경해 피처 벡터화 적용
tfidf_vect = TfidfVectorizer(stop_words = 'english', ngram_range = (1, 2), max_df = 300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)

print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.692


### GridSearchCV를 이용한 하이퍼 파라미터 최적화
- 로지스틱 회귀의 C 파라미터만 변경하면서 최적의 C 값을 찾은 뒤 이 C값으로 학습된 모델에서 테스트 데이터로 예측해 성능을 평가

In [7]:
from sklearn.model_selection import GridSearchCV

# 최적 C 값 도출 튜닝 수행, cv는 3 폴드 세트로 설정
params = {'C' : [0.01, 0.1, 1, 5, 10]}
grid_cv_lr = GridSearchCV(lr_clf, param_grid = params, cv = 3, scoring = 'accuracy', verbose = 1)
grid_cv_lr.fit(X_train_tfidf_vect, y_train)
print('Logistic Regression best C parameter: ', grid_cv_lr.best_params_)

# 최적 C 값으로 학습된 grid_cv로 예측 및 정확도 평가
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
Logistic Regression best C parameter:  {'C': 10}
TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.701


## 사이킷런 파이프라인 사용 및 GridsearchCV와의 결합

< 사이킷런의 pipeline 클래스 >
- 피처 벡터화와 ML 알고리즘 학습/예측을 위한 코드 작성을 한번에 진행할 수 있음
- 데이터 전처리와 머신러닝 학습 과정을 통일된 API 기반에서 처리할 수 있어 더 직관적인 ML 모델 코드를 생성할 수 있음
- 대용량 데이터의 피처 벡터화 결과를 별도 데이터로 저장하지 않고 스트림 기반에서 바로 머신러닝 알고리즘의 데이터로 입력 -> 수행시간 절약
- 텍스트 기반의 피처 벡터화뿐만 아니라 모든 데이터 전처리 작업과 Estimator를 결합할 수 있음

### pipeline 객체  선언
- 위에서 텍스트 분류 예제 코드를 pipeline을 이용해 다시 작성한 코드


```
tfidf_vect = TfidfVectorizer(stop_words = 'english', ngram_range = (1, 2), max_df = 300)
```


```
pipeline = Pipeline([('tfidf_vect', TfidfVectorizer(stop_words = 'english')), ('lr_clf', LogisticRegression(random_state = 156))])
```

TfidfVetorizer 객체를 tfidf_vect 객체 면수 명으로, LogisticRegression 객체를 lr_clf라는 객체 변수 명으로 생성 -> 이 두 개의 객체를 파이프 라인으로 연결하는 Pipeline 객체 pipeline을 생성한다는 의미



### pipeline 사용
- 기존 TfIdtVectorizer의 학습 데이터와 테스트 데이터에 대한 fit( )과 transform( ) 수행을 통한 피처 벡터화와 LogisticRegresor의 fit( )과 predict( ) 수행을 통한 머신러닝 모델의 학습과 예측이 Pipeline의 fit( )과 predict( )로 통일돼 수행
- 머신러닝 코드를 더 직관적이고 쉽게 작성

In [8]:
from sklearn.pipeline import Pipeline

# TfidfVectorizer 객체를 tfidf_vect로, LogisticRegression 객체를 lr_clf로 생성하는 Pipeline 생성
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words = 'english', ngram_range = (1, 2), max_df = 300)),
    ('lr_clf', LogisticRegression(C = 10))
])

# 별도의 TfidfVectorizer 객체의 fit( ), transform( )과 LogisticRegression의 fit( ), predict( )가 필요 없음
# pipeline의 fit( )과 predict( )만으로 한꺼번에 피처 벡터화와 ML 학습/예측이 가능
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

Pipeline을 통한 Logistic Regression의 예측 정확도는 0.701


## Pipeline 기반에서 하이퍼 파라미터 튜닝을 GridSearchCV 방식으로 진행
- GridSearchCV 클래스의 생성 파라미터로 Pipeline을 입력
- 피처 벡터화를 위한 파라미터와 ML 알고리즘의 하이퍼 파라미터를 모두 한번에 GridSearchCV를 이용해 최적화할 수 있음

- GridSearchCV에 pipeline을 입력하면서 TfidfVectorizer의 파라미터와 Logistic Regression의 하이퍼 파라미터를 함께 최적화
- GridSearchCV에 Estimator가 아닌 pipeline을 입력할 경우에는 param_grid의 입력 값 설정이 기존과 다름
  - 딕셔너리 형태의 key, value 값 가짐
  - value를 리스트 형태로 입력하는 것은 동일
  - key 값의 tidfvect_ngramrange와 같이 하이퍼 파라미터명이 객체 변수명과 결합 돼 제공
    - Pipeline을 GridSearchCV에 인자로 입력하면 GridSearchCV는 Pipeline을 구성하는 피처 벡터화 객체의 파라미터와 Estimator 객체의 하이퍼 파라미터를 각각 구별할 수 있어야 하는데 이때 개별 객체명과 파라미터명/하이퍼 파라미터명 을 결합해 Key값으로 할당
   - Tfdifvectorizer 객체 변수인 tidif_vect의 ngram_range 파라미터 값을 변화시키면서 최적화하기를 원한다면 객체 변수명인 tfidf_vect에 언더바 2개를 연달아 붙인 뒤 파라미터명인 ngram_range를 결합해 'tfidf_vect_ngram_range'를 key값으로 할당
- 피처 벡터화에 사용되는 파라미터와 GridSearchCV 하이퍼 파라미터를 합치면 최적화를 위한 너무 많은 경우의 수가 발생 => 많은 튜닝 시간 소모
  - 예제에서 pipeline + GridSearchCv 기반으로 하이퍼 파라미터 튜닝을 적용해 27개의 파라미터 경우의 수*3개의 CV로 81번의 학습과 검증을 수행

In [9]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words = 'english')),
    ('lr_clf', LogisticRegression())
])

# pipeline에 기술된 각각의 객체 변수에 언더바 2개를 연달아 붙여 GridSearchCV에 사용됨
# 파라미터/하이퍼 파라미터 이름과 값을 설정
params = {'tfidf_vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
          'tfidf_vect__max_df': [100, 300, 700],
          'lr_clf__C': [1, 5, 10]
          }

# GridSearchCV의 생성자에 Estimator가 아닌 pipeline 객체 입력
grid_cv_pipe = GridSearchCV(pipeline, param_grid = params, cv = 3, scoring = 'accuracy', verbose = 1)
grid_cv_pipe.fit(X_train, y_train)
print(grid_cv_pipe.best_params_, grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

Fitting 3 folds for each of 27 candidates, totalling 81 fits


KeyboardInterrupt: 